A test NN setup
1. Inputs: 1000 Group flux profile and number densities
2. Outputs: 1 GG XS (for 5 Reactions)

In [1]:
%matplotlib inline
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import os
import csv 

import openmc
import openmc.mgxs
import openmc.model

In [2]:
# Clear old data
! rm -rf flux_train.csv
! rm -rf n2n_train.csv
! rm -rf n3n_train.csv
! rm -rf flux_eval.csv
! rm -rf n2n_eval.csv
! rm -rf n3n_eval.csv

    MT = 16 (n,2n)
    MT = 17 (n,3n)
    MT = 37 (n,4n)
    MT = 103 (n,p)
    MT = 107 (n,a)

In [3]:
# Load XS data into object
path='/Users/zhuoranhan/MIT/nucleardata/endfb71_hdf5/'
filename='U238.h5'
u238 = openmc.data.IncidentNeutron.from_hdf5(path+filename)
# u238.reactions

In [4]:
def build_model(dilution):
    ##################
    ### material #####
    ##################
    ### dilution here is used to tune number densities
    u238_frac = 1/(dilution+1)
    h1_frac = dilution/(dilution+1)
    ppm = 1000
    homo = openmc.Material(name='homo')
    homo.add_nuclide('U238', u238_frac)
    homo.add_nuclide('H1', h1_frac)
    homo.add_element('B',ppm*1e-6)
    homo.set_density('sum')

    materials = openmc.Materials([homo])
    
    ##################
    ### geometry #####
    ##################
    pitch = 100
    xy_box = openmc.model.rectangular_prism(pitch, pitch, boundary_type='reflective')
    z0 = openmc.ZPlane(z0=-100, boundary_type='reflective')
    z1 = openmc.ZPlane(z0=100, boundary_type='reflective')

    root_cell = openmc.Cell(name='roo_cell', fill=homo)
    root_cell.region = xy_box & +z0 & -z1
    root = openmc.Universe()
    root.add_cell(root_cell)

    geometry = openmc.Geometry(root)
    
    ################
    ### settings ###
    ################
    settings = openmc.Settings()
    settings.run_mode = 'fixed source'
    settings.cutoff = {'energy_neutron': 1.0e-5}

    settings.batches = 100
    settings.inactive = 15
    settings.particles = 10000

    # uniform spatial distribution
    # Watt spectrum
    space = openmc.stats.Box((-pitch, -pitch, -pitch), (pitch, pitch, pitch), only_fissionable=True)
    
    # change source distribution to change specturm
    a = np.random.uniform(1e5,2e6)
    b = np.random.uniform(1e-6,1e-5)
    energy = openmc.stats.Watt(a,b)

    settings.source = openmc.Source(space=space,energy=energy)

    ################
    ### tallies ####
    ################
    tallies_file = openmc.Tallies()

    energy_bin= np.logspace(6, 8, num=1000)
    energy_filter = openmc.EnergyFilter(energy_bin)
    energy_range_filter = openmc.EnergyFilter([1e6,1e8])
    
    cell_filter = openmc.CellFilter(root_cell)

    flux_tally = openmc.Tally(name='flux')
    flux_tally.filters = [cell_filter]
    flux_tally.filters.append(energy_filter)
    flux_tally.scores = ['flux']
    tallies_file.append(flux_tally)
    
    n2n_tally = openmc.Tally(name='n2n')
    n2n_tally.filters = [cell_filter]
    n2n_tally.filters.append(energy_range_filter)
    n2n_tally.scores = ['(n,2n)']
    n2n_tally.nuclides = ['U238']
    tallies_file.append(n2n_tally)


    n3n_tally = openmc.Tally(name='n3n')
    n3n_tally.filters = [cell_filter]
    n3n_tally.filters.append(energy_range_filter)
    n3n_tally.scores = ['(n,3n)']
    n3n_tally.nuclides = ['U238']
    tallies_file.append(n3n_tally)

    model = openmc.model.Model(geometry, materials, settings, tallies_file)
    
    return model

In [5]:
def get_result():
#     sp_path='./'+str(dilution)+'/statepoint.100.h5'
    sp_path='./statepoint.100.h5'
    sp = openmc.StatePoint(sp_path)
    
#     flux tally
    flux_tally = sp.get_tally(name='flux')
    flux = flux_tally.get_values(scores=['flux'],value='mean')
    flux = flux.ravel()
    
#     n2n
    n2n_tally = sp.get_tally(name='n2n')
    n2n = n2n_tally.get_values(scores=['(n,2n)'],value='mean')
    n2n = n2n.ravel()
    
#   n3n
    n3n_tally = sp.get_tally(name='n3n')
    n3n = n3n_tally.get_values(scores=['(n,3n)'],value='mean')
    n3n = n3n.ravel()
    
    return flux, n2n, n3n

#### Run OpenMC and Save Data

In [6]:
## Training Data
max_iters = 100
dilution = 100
for i in range(max_iters):
    model = build_model(dilution)
    model.run(output=False)
    flux, n2n, n3n = get_result()

    with open('flux_train.csv','a',newline='') as ff:
        writer=csv.writer(ff)
        writer.writerow(flux)
    with open('n2n_train.csv','a',newline='') as n2nf:
        writer=csv.writer(n2nf)
        writer.writerow(n2n)
    with open('n3n_train.csv','a',newline='') as n3nf:
        writer=csv.writer(n3nf)
        writer.writerow(n3n)

In [7]:
### Evaluation Data
eval_iters = 10
dilution = 100
for i in range(eval_iters):
    openmc_model = build_model(dilution)
    openmc_model.run(output=False)
    flux, n2n, n3n = get_result()

    with open('flux_eval.csv','a',newline='') as ff:
        writer=csv.writer(ff)
        writer.writerow(flux)
    with open('n2n_eval.csv','a',newline='') as n2nf:
        writer=csv.writer(n2nf)
        writer.writerow(n2n)
    with open('n3n_eval.csv','a',newline='') as n3nf:
        writer=csv.writer(n3nf)
        writer.writerow(n3n)